# DeepAR - Experiments

### DeepAR (Salinas et al., 2017)

The **DeepAR** algorithm uses autoregressive features that are used as the inputs to a recurrent neural network. The RNN uses LSTM cells as default but can also changed to GRU cells. In both cases previous time points are taken as input. 

- [`DeepAREestimator`](https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html)

In [1]:
import gluonts
gluonts.__version__

'0.3.4.dev77+g197866b'

In [2]:
# !pip install gluonts

In [3]:
# standard imports
import numpy as np
import pandas as pd

# json
import json

# gluon data
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# model imports
# from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint


INFO:root:Using CPU


In [4]:
def deepar(data="m4_monthly", seed=42, epochs=100, batches=50):
    
    dataset = get_dataset(data, regenerate=False)
    mx.random.seed(seed)
    np.random.seed(seed)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,      
        num_batches_per_epoch=batches,      
        learning_rate=1e-3,
    )
    
    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        cardinality=[cardinality],
#         context_length=dataset.metadata.prediction_length,
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True   
    )
        
    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    output["seed"] = seed
    return(output)


In [5]:
%%time
res = deepar(data="m4_daily", seed=42, epochs=100, batches=200)
pprint(res)

INFO:root:downloading and processing m4_daily


saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily/train/data.json
saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily/test/data.json


INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 200/200 [00:09<00:00, 21.14it/s, avg_epoch_loss=6.87]
INFO:root:Epoch[0] Elapsed time 9.461 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.865995
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:09<00:00, 20.95it/s, avg_epoch_loss=6.44]
INFO:root:Epoch[1] Elapsed time 9.548 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.439188
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:09<00:00, 20.03it/s, avg_epoch_loss=6.25]
INFO:root:Epoch[2] Elapsed time 9.988 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.251020
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:09<00:00, 21.08it/s, avg_epoch_loss=6.14]
INFO:root:Epoch[3] Elapsed time 9.489 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=6.141487
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 200/200 [00:09<00:00, 20.90it/s, avg_e

{'MASE': 5.02448342,
 'MSIS': 72.47790163,
 'epochs': 100,
 'sMAPE': 0.04432468,
 'seed': 42,
 'wQuantileLoss[0.5]': 0.04171161,
 'wQuantileLoss[0.9]': 0.02448477}
CPU times: user 33min 40s, sys: 35.1 s, total: 34min 16s
Wall time: 19min 46s


In [8]:
%%time
res = deepar(data="m4_daily", seed=43, epochs=150, batches=200)
pprint(res)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 200/200 [00:09<00:00, 21.65it/s, avg_epoch_loss=7.27]
INFO:root:Epoch[0] Elapsed time 9.240 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.272206
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:09<00:00, 20.83it/s, avg_epoch_loss=6.4]
INFO:root:Epoch[1] Elapsed time 9.601 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.398123
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:09<00:00, 20.64it/s, avg_epoch_loss=6.26]
INFO:root:Epoch[2] Elapsed time 9.689 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.259896
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:09<00:00, 20.66it/s, avg_epoch_loss=6.19]
INFO:root:Epoch[3] Elapsed time 9.682 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=6.18593

{'MASE': 3.46143932,
 'MSIS': 41.56383944,
 'epochs': 150,
 'sMAPE': 0.03259256,
 'seed': 43,
 'wQuantileLoss[0.5]': 0.02934965,
 'wQuantileLoss[0.9]': 0.01296957}
CPU times: user 47min 13s, sys: 43.6 s, total: 47min 56s
Wall time: 27min 3s


In [9]:
%%time
res = deepar(data="m4_daily", seed=44, epochs=200, batches=200)
pprint(res)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 200/200 [00:09<00:00, 21.35it/s, avg_epoch_loss=6.85]
INFO:root:Epoch[0] Elapsed time 9.370 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.847021
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:09<00:00, 20.73it/s, avg_epoch_loss=6.39]
INFO:root:Epoch[1] Elapsed time 9.651 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.387433
INFO:root:Epoch[2] Learning rate is 0.001
  0%|          | 0/200 [00:06<?, ?it/s, avg_epoch_loss=6.21]


KeyboardInterrupt: 

In [12]:
# %%time
# results = []

# if __name__ == "__main__":
#     for i in range(42, 45):
#         print("Seed:", i)
#         res = deepar(data="m4_monthly", seed=i, epochs=100, batches=1000)
#         pprint(res)
#         results.append(res)
    
# results

### DeepAR - use_feat_static==False

In [7]:
def deepar_no_feat(data="m4_monthly", seed=42, epochs=100, batches=50):
    
    dataset = get_dataset(data, regenerate=False)
    mx.random.seed(seed)
    np.random.seed(seed)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,      
        num_batches_per_epoch=batches,      
        learning_rate=1e-3,
    )
    
#     cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
#         cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
#         use_feat_static_cat=True   
    )
        
    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    output["seed"] = seed
    return(output)


In [8]:
%%time
res = deepar_no_feat(data="m4_monthly", seed=42, epochs=100, batches=1000)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_monthly.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 13463
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 1000/1000 [01:11<00:00, 13.97it/s, avg_epoch_loss=7.03]
INFO:root:Epoch[0] Elapsed time 71.576 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.031377
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 1000/1000 [01:09<00:00, 14.45it/s, avg_epoch_loss=6.9]
INFO:root:Epoch[1] Elapsed time 69.187 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.904093
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 1000/1000 [01:09<00:00, 14.46it/s, avg_epoch_loss=6.62]
INFO:root:Epoch[2] Elapsed time 69.186 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.615703
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 1000/1000 [01:09<00:00, 14.47it/s, avg_epoch_loss=6.62]
INFO:root:Epoch[3] Elapsed tim

{'MASE': 0.9894242,
 'MSIS': 9.79866835,
 'epochs': 100,
 'sMAPE': 0.13057258,
 'seed': 42,
 'wQuantileLoss[0.5]': 0.11604762,
 'wQuantileLoss[0.9]': 0.06786353}
Wall time: 11h 55min 14s
